In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import UnexpectedAlertPresentException
import requests
import datetime

In [65]:
def movie_crawling(webdriver_path, service, driver, id):
    webdriver_path = webdriver_path
    service = service
    driver = driver
    try:
        url = f'http://cine21.com/movie/info/?movie_id={id}'
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        sub_info = soup.find_all('p', class_='sub_info')
        sub_info_list = [i.text.strip().replace('\n', ' ').replace('\t', ' ') for i in sub_info]
        ko_name = soup.find('p', class_='tit').text
        en_name = soup.find('p', class_='tit_eng').text.strip().split('\n')[-1]
        story = soup.find('div', class_='story').text.strip().replace('\t', '').replace('\n', ' ').replace('  ', ' ')
        if story == '':
            story = None
        year = sub_info_list[0].split()[0]
        country = sub_info_list[0].split()[1]
        poster_link = soup.find('div', class_='mov_poster').find('img')['src']
        if poster_link == 'https://image.cine21.com/resize//noimg[X230,330].jpg':
            poster_link = None

        try:
            if sub_info_list[2][:3] == '개봉일' and len(sub_info_list[2][6:16]) == 10:
                pd.to_datetime(sub_info_list[2][6:16])
                release_date = sub_info_list[2][6:16]
            else:
                release_date = None
        except:
            release_date = None
    
        if sub_info_list[2][17:21] == '누적관객':
            audience = int(sub_info_list[2][24:].replace(',', '').replace('명', '').replace(',', ''))
        else:
            audience = None
        
        if sub_info_list[1].split('상영시간 : ')[0] != '':
            genre = sub_info_list[1].split('상영시간 : ')[0][:-1]
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])
        else:
            genre = None
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])

        if sub_info_list[3][0:2] == '감독':
            director = sub_info_list[3][18:]
        else:
            director = None

        for i in range(1, 6):
            if soup.find('span', class_=f'grade00{i}') != None:
                grade = soup.find('span', class_=f'grade00{i}').text
                break
            elif i == 5:
                grade = None

        return [id, ko_name, en_name, year, country, grade, genre, running_time, release_date, audience, director, story, poster_link]
    
    except UnexpectedAlertPresentException as e1:
        return e1.alert_text
    
    except IndexError as e2:
        return '영화 정보 불충분'
    
    except AttributeError as e3:
        return '일시적으로 서비스 장애가 발생하였습니다.'

In [66]:
create_table_query = """
CREATE TABLE IF NOT EXISTS movies (
    id INT PRIMARY KEY,
    ko_name VARCHAR(255),
    en_name VARCHAR(255),
    country VARCHAR(255),
    grade VARCHAR(255),
    genre VARCHAR(255),
    runtime INT,
    release_date DATE,
    audience INT,
    director VARCHAR(255),
    story VARCHAR(255),
    poster_link VARCHAR(255))"""

In [63]:
# # My SQL 데이터베이스 연결
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# cursor = conn.cursor()

# cursor.execute(create_table_query)
# conn.commit()
# print("테이블이 성공적으로 생성되었습니다")

# conn.close()

In [103]:
# # My SQL 데이터베이스 연결
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# # 연결 확인
# if conn.is_connected():
#     print("MySQL에 성공적으로 연결되었습니다")
# else:
#     print("MySQL 연결 실패")

# cursor = conn.cursor()

# webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
# service = Service(webdriver_path)
# driver = webdriver.Chrome(service=service)

# insert_query = """
# INSERT INTO movie
# VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

# for i in range(61826, 70000):
#     data = movie_crawling(webdriver_path, service, driver, i)
#     if data == '존재하지 않는 영화입니다.':
#         print(data)
#         continue
#     elif data == '성인용 컨텐츠입니다. 로그인 해 주세요.':
#         print(data)
#         continue
#     elif data == '영화 정보 불충분':
#         print(data)
#         continue
#     elif data == '일시적으로 서비스 장애가 발생하였습니다.':
#         print(data)
#         continue
#     cursor.execute(insert_query, data)
#     conn.commit()
#     print(data)

In [ ]:
# # MySQL 연결 설정
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# # 커서 객체 생성
# cursor = conn.cursor()

# # SQL 쿼리문
# query = """
# SELECT * FROM movie
# WHERE (poster_link IS NOT NULL) and (genre IS NOT NULL);
# """

# # SQL 쿼리 실행
# df = pd.read_sql(query, conn)

# # 데이터프레임 csv로 로컬에 저장
# df.to_csv('movieDF.csv', index=False)